In [ ]:
!wget https://thisanimedoesnotexist.ai/downloads/wd-v1-2-full-ema.ckpt

In [ ]:
!git clone https://github.com/CompVis/stable-diffusion.git


In [3]:
%cd stable-diffusion

/workspace/stable-diffusion


In [4]:
#copy requirements.txt from https://github.com/justinpinkney/stable-diffusion/blob/main/requirements.txt
!pip install -r requirements.txt

Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 18))
  Updating ./src/taming-transformers clone (to revision master)
  Running command git fetch -q --tags
  Running command git reset --hard -q 24268930bf1dce879235a7fddd0b2355b84d7ea6
Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip (from -r requirements.txt (line 19))
  Updating ./src/clip clone (to revision main)
  Running command git fetch -q --tags
  Running command git reset --hard -q d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
Obtaining file:///workspace/stable-diffusion (from -r requirements.txt (line 20))
  Using cached albumentations-0.4.3-py3-none-any.whl
  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)
  Using cached pudb-2019.2-py3-none-any.whl
  Using cached imageio_ffmpeg-0.4.2-py3-none-manylinux2010_x86_64.whl (26.9 MB)
  Using cached omegacon

In [ ]:
!pip install --upgrade pytorch-lightning

In [2]:
from io import BytesIO
import os
from contextlib import nullcontext

import fire
import numpy as np
import torch
from einops import rearrange
from omegaconf import OmegaConf
from PIL import Image
from torch import autocast
from torchvision import transforms
import requests

from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from ldm.util import instantiate_from_config

In [3]:
from pytorch_lightning import seed_everything

In [11]:
!apt-get update -y && apt-get install libgl1 -y


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Reading package lists... Done                      
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libelf1 libgl1-mesa-dri libglapi-mesa libglvnd0 libglx-mesa0 libglx0
  libllvm10 libpciaccess0 libsensors4 libx11-6 libx11-data libx11-xcb1 libxau6
  libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0 libxcb-sync1 libxcb1
  libxdamage1 libxdmcp6 libxext6 libxfixes3 libxshmfence1 libxxf86vm1
Suggested packages:
  pciutils lm-sensors
The following NEW packages will be installed:
  libdrm-amdgpu1 libdrm-common l

In [16]:
import argparse, os, sys, glob
import cv2
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid
import time
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import contextmanager, nullcontext

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libglib2.0-data libicu60 libxml2 shared-mime-info xdg-user-dirs
The following NEW packages will be installed:
  libglib2.0-0 libglib2.0-data libicu60 libxml2 shared-mime-info xdg-user-dirs
0 upgraded, 6 newly installed, 0 to remove and 6 not upgraded.
Need to get 10.4 MB of archives.
After this operation, 40.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libglib2.0-0 amd64 2.56.4-0ubuntu0.18.04.9 [1169 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libglib2.0-data all 2.56.4-0ubuntu0.18.04.9 [4728 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libicu60 amd64 60.2-3ubuntu3.2 [8050 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxml2 amd64 2.9.4+dfsg1-6.1ubuntu1.7 [663 kB]
Get:5 http://archive.ubuntu.com/ubuntu b

In [18]:
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

In [20]:
config = OmegaConf.load("configs/stable-diffusion/v1-inference.yaml")
model = load_model_from_config(config, f"../wd-v1-2-full-ema.ckpt")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

sampler = DDIMSampler(model)

sample_path = "outs"
os.makedirs(sample_path, exist_ok=True)

start_code = None

Loading model from ../wd-v1-2-full-ema.ckpt
Global Step: 487750
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.16.self_attn.out_proj.bias', 'vision_model.encoder.layers.10.self_attn.k_proj.bias', 'vision_model.encoder.layers.3.layer_norm1.weight', 'vision_model.encoder.layers.9.layer_norm1.weight', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.19.mlp.fc1.bias', 'vision_model.encoder.layers.21.self_attn.k_proj.bias', 'vision_model.encoder.layers.19.self_attn.out_proj.bias', 'vision_model.encoder.layers.6.self_attn.k_proj.weight', 'vision_model.encoder.layers.23.mlp.fc2.weight', 'vision_model.encoder.layers.13.mlp.fc2.weight', 'vision_model.encoder.layers.15.layer_norm1.bias', 'vision_model.encoder.layers.12.layer_norm1.bias', 'vision_model.encoder.layers.5.self_attn.q_proj.weight', 'vision_model.encoder.layers.7.layer_norm2.bias', 'vision_model.encoder.layers.14.self_attn.v_proj.weight', 'vision_model.encoder.layers.12

In [25]:
import random

In [49]:
seeds = [random.randint(1, 2000000) for i in range(10)]

In [51]:
for seed in seeds[:7]:
    batch_size = 1

    prompt = "holo from spice and wolf learning to code, highres"
    scale = 7.5
    C = 4
    H = 512
    W = 512
    f = 8
    data = [batch_size * [prompt]]

    seed_everything(seed)

    precision_scope = autocast
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                all_samples = list()
                for n in trange(1, desc="Sampling"):
                    for prompts in tqdm(data, desc="data"):
                        uc = None
                        if scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)
                        shape = [C, H // f, W // f]
                        samples_ddim, _ = sampler.sample(S=50,
                                                         conditioning=c,
                                                         batch_size=1,
                                                         shape=shape,
                                                         verbose=False,
                                                         unconditional_guidance_scale=scale,
                                                         unconditional_conditioning=uc,
                                                         eta=0.0,
                                                         x_T=start_code)

                        x_samples_ddim = model.decode_first_stage(samples_ddim)
                        x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                        x_samples_ddim = x_samples_ddim.cpu().permute(0, 2, 3, 1).numpy()

                        x_checked_image_torch = torch.from_numpy(x_samples_ddim).permute(0, 3, 1, 2)

                        for x_sample in x_checked_image_torch:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            img = Image.fromarray(x_sample.astype(np.uint8))
                            img.save(os.path.join(sample_path, f"{prompt[:25]}_{seed}.png"))

                toc = time.time()

Global seed set to 793339
data:   0%|          | 0/1 [00:00<?, ?it/s]

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps




DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler:   2%|▏         | 1/50 [00:00<00:12,  3.95it/s]

DDIM Sampler:   4%|▍         | 2/50 [00:00<00:08,  5.83it/s]

DDIM Sampler:   8%|▊         | 4/50 [00:00<00:05,  7.99it/s]

DDIM Sampler:  10%|█         | 5/50 [00:00<00:05,  8.36it/s]

DDIM Sampler:  14%|█▍        | 7/50 [00:00<00:04,  9.25it/s]

DDIM Sampler:  16%|█▌        | 8/50 [00:00<00:04,  9.20it/s]

DDIM Sampler:  18%|█▊        | 9/50 [00:01<00:04,  9.12it/s]

DDIM Sampler:  20%|██        | 10/50 [00:01<00:04,  9.12it/s]

DDIM Sampler:  22%|██▏       | 11/50 [00:01<00:04,  9.07it/s]

DDIM Sampler:  24%|██▍       | 12/50 [00:01<00:04,  9.04it/s]

DDIM Sampler:  28%|██▊       | 14/50 [00:01<00:03, 10.05it/s]

DDIM Sampler:  32%|███▏      | 16/50 [00:01<00:03, 10.66it/s]

DDIM Sampler:  36%|███▌      | 18/50 [00:01<00:02, 10.74it/s]

DDIM Sampler:  40%|████      | 20/50 [00:02<00:02, 10.50it/s]

DDIM Sampler:  44%|████▍     | 22/50 [00:02<00:02, 10.88it/s]

DDIM S

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps




DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler:   2%|▏         | 1/50 [00:00<00:12,  3.81it/s]

DDIM Sampler:   4%|▍         | 2/50 [00:00<00:08,  5.72it/s]

DDIM Sampler:   8%|▊         | 4/50 [00:00<00:05,  8.37it/s]

DDIM Sampler:  12%|█▏        | 6/50 [00:00<00:04,  9.61it/s]

DDIM Sampler:  16%|█▌        | 8/50 [00:00<00:04, 10.09it/s]

DDIM Sampler:  20%|██        | 10/50 [00:01<00:03, 10.61it/s]

DDIM Sampler:  24%|██▍       | 12/50 [00:01<00:03, 10.48it/s]

DDIM Sampler:  28%|██▊       | 14/50 [00:01<00:03, 10.45it/s]

DDIM Sampler:  32%|███▏      | 16/50 [00:01<00:03, 10.35it/s]

DDIM Sampler:  36%|███▌      | 18/50 [00:01<00:03, 10.36it/s]

DDIM Sampler:  40%|████      | 20/50 [00:02<00:02, 10.72it/s]

DDIM Sampler:  44%|████▍     | 22/50 [00:02<00:02, 10.64it/s]

DDIM Sampler:  48%|████▊     | 24/50 [00:02<00:02, 10.93it/s]

DDIM Sampler:  52%|█████▏    | 26/50 [00:02<00:02, 11.15it/s]

DDIM Sampler:  56%|█████▌    | 28/50 [00:02<00:01, 11.05it/s]

DDIM

...



DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler:   2%|▏         | 1/50 [00:00<00:09,  5.28it/s]

DDIM Sampler:   4%|▍         | 2/50 [00:00<00:06,  6.92it/s]

DDIM Sampler:   6%|▌         | 3/50 [00:00<00:06,  7.70it/s]

DDIM Sampler:   8%|▊         | 4/50 [00:00<00:05,  8.14it/s]

DDIM Sampler:  12%|█▏        | 6/50 [00:00<00:04,  9.17it/s]

DDIM Sampler:  16%|█▌        | 8/50 [00:00<00:04, 10.09it/s]

DDIM Sampler:  20%|██        | 10/50 [00:01<00:03, 10.32it/s]

DDIM Sampler:  24%|██▍       | 12/50 [00:01<00:03, 10.43it/s]

DDIM Sampler:  28%|██▊       | 14/50 [00:01<00:03, 10.50it/s]

DDIM Sampler:  32%|███▏      | 16/50 [00:01<00:03, 10.52it/s]

DDIM Sampler:  36%|███▌      | 18/50 [00:01<00:03, 10.50it/s]

DDIM Sampler:  40%|████      | 20/50 [00:02<00:02, 10.78it/s]

DDIM Sampler:  44%|████▍     | 22/50 [00:02<00:02, 10.60it/s]

DDIM Sampler:  48%|████▊     | 24/50 [00:02<00:02, 10.46it/s]

DDIM Sampler:  52%|█████▏    | 26/50 [00:02<00:02, 10.79it/s]

DDIM 

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps




DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler:   2%|▏         | 1/50 [00:00<00:12,  3.80it/s]

DDIM Sampler:   4%|▍         | 2/50 [00:00<00:08,  5.80it/s]

DDIM Sampler:   8%|▊         | 4/50 [00:00<00:05,  8.00it/s]

DDIM Sampler:  12%|█▏        | 6/50 [00:00<00:04,  9.41it/s]

DDIM Sampler:  14%|█▍        | 7/50 [00:00<00:04,  9.27it/s]

DDIM Sampler:  16%|█▌        | 8/50 [00:00<00:04,  9.17it/s]

DDIM Sampler:  18%|█▊        | 9/50 [00:01<00:04,  9.10it/s]

DDIM Sampler:  20%|██        | 10/50 [00:01<00:04,  9.06it/s]

DDIM Sampler:  24%|██▍       | 12/50 [00:01<00:03, 10.09it/s]

DDIM Sampler:  28%|██▊       | 14/50 [00:01<00:03, 10.67it/s]

DDIM Sampler:  32%|███▏      | 16/50 [00:01<00:03, 11.03it/s]

DDIM Sampler:  36%|███▌      | 18/50 [00:01<00:02, 10.72it/s]

DDIM Sampler:  40%|████      | 20/50 [00:02<00:02, 10.58it/s]

DDIM Sampler:  44%|████▍     | 22/50 [00:02<00:02, 10.57it/s]

DDIM Sampler:  48%|████▊     | 24/50 [00:02<00:02, 10.33it/s]

DDIM S

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps




DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler:   2%|▏         | 1/50 [00:00<00:09,  5.28it/s]

DDIM Sampler:   4%|▍         | 2/50 [00:00<00:06,  6.87it/s]

DDIM Sampler:   6%|▌         | 3/50 [00:00<00:06,  7.62it/s]

DDIM Sampler:   8%|▊         | 4/50 [00:00<00:05,  8.03it/s]

DDIM Sampler:  10%|█         | 5/50 [00:00<00:05,  8.28it/s]

DDIM Sampler:  12%|█▏        | 6/50 [00:00<00:05,  8.44it/s]

DDIM Sampler:  16%|█▌        | 8/50 [00:00<00:04,  9.75it/s]

DDIM Sampler:  18%|█▊        | 9/50 [00:01<00:04,  9.49it/s]

DDIM Sampler:  20%|██        | 10/50 [00:01<00:04,  9.30it/s]

DDIM Sampler:  22%|██▏       | 11/50 [00:01<00:04,  9.14it/s]

DDIM Sampler:  24%|██▍       | 12/50 [00:01<00:04,  9.03it/s]

DDIM Sampler:  28%|██▊       | 14/50 [00:01<00:03,  9.86it/s]

DDIM Sampler:  32%|███▏      | 16/50 [00:01<00:03, 10.45it/s]

DDIM Sampler:  36%|███▌      | 18/50 [00:01<00:03, 10.32it/s]

DDIM Sampler:  40%|████      | 20/50 [00:02<00:02, 10.22it/s]

DDIM Sa

In [5]:
%cd stable-diffusion

/workspace/stable-diffusion
